In [1]:
import pandas as pd
import openai
import numpy as np
import tiktoken
import time
import matplotlib.pyplot as plt
openai.api_key = ''

In [2]:
import pandas as pd

def input_loop():
    # Initialize an empty DataFrame to act as the database
    database = pd.DataFrame(columns=['date', 'author', 'text'])

    while True:
        # Ask for input
        date = input('Enter a date: ')
        author = input('Enter an author: ')
        text = input('Enter some text: ')

        # Add the inputs to the DataFrame
        data = pd.DataFrame([[date, author, text]], columns=['date', 'author', 'text'])
        database = pd.concat([database, data], ignore_index=True)

        # Save the DataFrame to a CSV file
        database.to_csv('database2.csv', index=False)

        # Ask if the user wants to continue
        cont = input('Do you want to continue? (y/n)')
        if cont.lower() != 'y':
            break

    return database


In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Scraping the speeches of the fed

url = 'https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

links = soup.find_all('a', attrs={'href': True})
yearly_links_raw = []

for link in links:
    href = link['href']
    if 'fomchistorical' in href and "htm" in href:
        yearly_links_raw.append(f"https://www.federalreserve.gov{href}")


In [26]:
yearly_links_raw = yearly_links_raw[0:12]

In [27]:
yearly_links_raw

['https://www.federalreserve.gov/monetarypolicy/fomchistorical2017.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2016.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2015.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2014.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2013.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2012.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2011.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2010.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2009.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2008.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2007.htm',
 'https://www.federalreserve.gov/monetarypolicy/fomchistorical2006.htm']

In [45]:
def get_speeches_from_yearly_page(yearly_url):
    response = requests.get(yearly_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    speech_links = []
    for link in soup.find_all("a"):
        url = link.get("href", "")
        if 'monetary20' in url and "a.htm" in url:
            speech_links.append({
                "url": f"https://www.federalreserve.gov{url}",
            })
        elif 'monetary/20' in url and "a.htm" in url:
            speech_links.append({
                "url": f"https://www.federalreserve.gov{url}",
            })

    return speech_links

In [46]:
speech_links = []
for yearly_url in yearly_links_raw:
    speech_links.extend(get_speeches_from_yearly_page(yearly_url))

In [50]:
speech_links

[{'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170201a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170315a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170503a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170614a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170726a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20170920a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171101a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171213a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20160127a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20160316a.htm'},
 {'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20160427a.htm'},

In [51]:
from datetime import datetime
speeches = []
for link in speech_links:
    
    url = link["url"]
    # Make a GET request to the webpage
    response = requests.get(url)

    # Parse the document using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract author and date from the page using their respective classes
    date = soup.find(class_='article__time').text.strip()
    date = datetime.strptime(date, "%B %d, %Y")

    # Extract speech from the page until <hr align="left" size="1" width="33%"> is encountered
    speech_parts = []
    for tag in soup.find('div', {'class': 'col-xs-12 col-sm-8 col-md-8'}).children:
        if tag.name == 'hr' and tag.get('align') == 'left' and tag.get('size') == '1' and tag.get('width') == '33%':
            break
        if tag.name == 'p':
            speech_parts.append(tag.text)

    speech = ' '.join(speech_parts)

    speeches.append({
        "date": date,
        "text": speech,
        "author": "FOMC"
    })

In [66]:
speeches_df1 = pd.DataFrame(speeches)
# Convert 'date' column to datetime, if it isn't already
speeches_df1['date'] = pd.to_datetime(speeches_df1['date']) #to remove all voting for the monetary policy action section

In [67]:
speeches_df1

,date,text,author
0,2017-02-01,Information received since the Federal Open Ma...,FOMC
1,2017-03-15,Information received since the Federal Open Ma...,FOMC
2,2017-05-03,Information received since the Federal Open Ma...,FOMC
3,2017-06-14,Information received since the Federal Open Ma...,FOMC
4,2017-07-26,Information received since the Federal Open Ma...,FOMC
...,...,...,...
94,2006-06-29,The Federal Open Market Committee decided toda...,FOMC
95,2006-08-08,The Federal Open Market Committee decided toda...,FOMC
96,2006-09-20,The Federal Open Market Committee decided toda...,FOMC
97,2006-10-25,The Federal Open Market Committee decided toda...,FOMC


In [78]:
speeches_df2 = pd.read_csv('fomc_speeches_df1.csv')
speeches_df2['date'] = pd.to_datetime(speeches_df2['date']) #to remove all voting for the monetary policy action section

In [87]:
complete = pd.concat([speeches_df2, speeches_df1], ignore_index=True)

In [89]:
complete.to_csv('fomc_speeches.csv', index=False)